In [6]:
! sudo pip install spicy

In [4]:
! sudo pip install pingouin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=964d8de165b461c9191e2a1db5c131a8786745d89071a36340ba25052f37c15d
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [16]:
import pandas as pd # leitura e manipulacao de arquivos
import numpy as np # analise numerica
import seaborn as sns # analise grafica
import matplotlib.pyplot as plt # analise grafica

In [9]:
import pingouin as pg
from spicy import stats as st

##Dados

In [17]:
# dados
dados_tratamentos = pd.read_csv('dados_tratamentos.csv', sep=';')
dados_bolsa = pd.read_csv('dados_bolsa.csv', sep=';', decimal=',')
dados_nps = pd.read_csv('nps_example.csv', sep=';')

In [13]:
dados_tratamentos.head()

,id,tratamento,evolucao
0,1,tratado,melhorou
1,2,tratado,melhorou
2,3,nao-tratado,melhorou
3,4,tratado,melhorou
4,5,tratado,nao-melhorou


In [18]:
dados_bolsa.head()

,data,petr4,bbdc3,vale5,ambv4,itub4
0,2004-05-26,-0.002270,0.009524,0.013699,0.032668,0.009843
1,2004-05-27,0.026301,0.028396,0.028303,0.014060,0.021442
2,2004-05-28,-0.016484,-0.000092,-0.025261,0.019151,0.000000
3,2004-05-31,0.010986,-0.009174,0.011235,-0.001786,0.000000
4,2004-06-01,0.028142,0.023056,-0.004518,0.013629,0.012107


In [15]:
dados_nps.head()

,id,response_status,how_long_listening,age,nps_score,gender
0,11706300,Complete,Less than 6 months,25-34,10.0,Female
1,11706302,Complete,1 year to less than 3 years,25-34,10.0,Female
2,11706307,Complete,6 months to less than a year,35-44,10.0,Female
3,11706312,Complete,Less than 6 months,35-44,10.0,Female
4,11706316,Complete,6 months to less than a year,25-34,10.0,Male


##Phi

In [19]:
# crosstab- criar tabelas cruzadas
tabela = pd.crosstab(dados_tratamentos['tratamento'],
                     dados_tratamentos['evolucao'])
tabela

evolucao,melhorou,nao-melhorou
tratamento,,
nao-tratado,26,29
tratado,35,15


In [28]:
# obter qui-quadrado
qui_quadrado = st.chi2_contingency(tabela)[0]
qui_quadrado

4.6625668947297125

In [31]:
# tamanho da amostra
n = np.sum(np.sum(tabela))
n

105

In [32]:
# obter o Phi
Phi = np.sqrt(qui_quadrado / n)
Phi

0.21072588592155886

##Teste de Qui-Quadrado

In [38]:
qui, p, gl, t_esp = st.chi2_contingency(tabela)
qui, p

(4.6625668947297125, 0.030827072412198585)

Temos evidencia para rejeitar a hipotese nula, uma vez que o p-value resultou um valor menor que 0.05

##V de cramer

In [42]:
# limpeza nos dados
dados_nps_filtrados = dados_nps[(dados_nps['response_status'] == 'Complete') & (dados_nps['nps_score'].notna()) & (dados_nps['gender'].notna())].reset_index()

In [44]:
# criar grupos
def create_nps_groups (x):
  if x <= 5:
    return 'detratores'
  elif x >= 9:
    return 'promotores'
  else:
    return 'neutros'

In [48]:
# aplicar a função
dados_nps_filtrados['nps_groups'] = dados_nps_filtrados['nps_score'].apply(lambda x: create_nps_groups(x))

In [50]:
# verificando a função
dados_nps_filtrados.groupby('nps_groups').size().to_frame('n')

,n
nps_groups,
detratores,25
neutros,171
promotores,2047


In [53]:
# criando tabela cruzada
tabela = pd.crosstab(dados_nps_filtrados['gender'],
                     dados_nps_filtrados['nps_groups'])
tabela

nps_groups,detratores,neutros,promotores
gender,,,
Female,12,105,1565
Male,13,66,482


In [54]:
# v de cramer método 1
vcramer_1 = st.contingency.association(tabela, method='cramer')
vcramer_1

0.1134646600588705

In [55]:
# v de cramer método 2
# obter valor do qui quadrado
qui_quadrado = st.chi2_contingency(tabela)[0]

0.1134646600588705

In [60]:
# tamanho da amostra
n = np.sum(np.sum(tabela))
n

2243

In [61]:
# minimo entre r e c
minshape = min(tabela.shape)-1
minshape

1

In [59]:
# v de cramer
vcramer_2 = np.sqrt(qui_quadrado / (n * minshape))
vcramer_2

0.1134646600588705

In [56]:
# teste qui-quadrado
qui, p, gl, t_esp = st.chi2_contingency(tabela)
qui, p

(28.87689583154292, 5.363666207418583e-07)

Temos evidencia para rejeitar a hipotese nula, uma vez que o p-value resultou um valor menor que 0.05

##Correlação

In [62]:
# verificar dados nulos
dados_bolsa[dados_bolsa.isnull()]

,data,petr4,bbdc3,vale5,ambv4,itub4
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1773,NaN,NaN,NaN,NaN,NaN,NaN
1774,NaN,NaN,NaN,NaN,NaN,NaN
1775,NaN,NaN,NaN,NaN,NaN,NaN
1776,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
# tratar os dados nulos
dados_bolsa_filtrados = dados_bolsa.dropna()
dados_bolsa_filtrados

,data,petr4,bbdc3,vale5,ambv4,itub4
0,2004-05-26,-0.002270,0.009524,0.013699,0.032668,0.009843
1,2004-05-27,0.026301,0.028396,0.028303,0.014060,0.021442
2,2004-05-28,-0.016484,-0.000092,-0.025261,0.019151,0.000000
3,2004-05-31,0.010986,-0.009174,0.011235,-0.001786,0.000000
4,2004-06-01,0.028142,0.023056,-0.004518,0.013629,0.012107
...,...,...,...,...,...,...
1773,2011-03-14,0.003905,-0.000407,-0.002139,-0.010953,0.004765
1774,2011-03-15,-0.008487,0.004473,-0.016724,-0.004873,0.011994
1775,2011-03-16,-0.009272,-0.017409,-0.025294,-0.020699,-0.023705
1776,2011-03-17,0.010799,-0.001236,0.027293,-0.001136,-0.005364




*   Pearson



In [65]:
# correlacao
pearson_correl = dados_bolsa_filtrados.corr()
pearson_correl

<ipython-input-65-545e8d9cee13>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  pearson_correl = dados_bolsa_filtrados.corr()


,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.539247,0.724023,0.392074,0.593834
bbdc3,0.539247,1.000000,0.592143,0.470529,0.778506
vale5,0.724023,0.592143,1.000000,0.482919,0.642838
ambv4,0.392074,0.470529,0.482919,1.000000,0.488886
itub4,0.593834,0.778506,0.642838,0.488886,1.000000


In [69]:
# matriz ajustada
pearson_correl.style.background_gradient(cmap='coolwarm').set_precision(2)

<ipython-input-69-254f00349427>:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  pearson_correl.style.background_gradient(cmap='coolwarm').set_precision(2)


,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.00,0.54,0.72,0.39,0.59
bbdc3,0.54,1.00,0.59,0.47,0.78
vale5,0.72,0.59,1.00,0.48,0.64
ambv4,0.39,0.47,0.48,1.00,0.49
itub4,0.59,0.78,0.64,0.49,1.00


In [72]:
# colunas para correlacionar
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']
# teste
pg.pairwise_corr(dados_bolsa_filtrados,
                columns = correl_columns,
                method = 'pearson')

,X,Y,method,alternative,n,r,CI95%,p-unc,BF10,power
0,petr4,bbdc3,pearson,two-sided,1604,0.539247,"[0.5, 0.57]",1.094616e-121,7.488e+117,1.0
1,petr4,vale5,pearson,two-sided,1604,0.724023,"[0.7, 0.75]",1.122582e-260,3.654e+256,1.0
2,petr4,ambv4,pearson,two-sided,1604,0.392074,"[0.35, 0.43]",4.390573e-60,3.057e+56,1.0
3,petr4,itub4,pearson,two-sided,1604,0.593834,"[0.56, 0.62]",1.793002e-153,3.791e+149,1.0
4,bbdc3,vale5,pearson,two-sided,1604,0.592143,"[0.56, 0.62]",2.143940e-152,3.189e+148,1.0
5,bbdc3,ambv4,pearson,two-sided,1604,0.470529,"[0.43, 0.51]",3.714265e-89,2.774e+85,1.0
6,bbdc3,itub4,pearson,two-sided,1604,0.778506,"[0.76, 0.8]",0.000000e+00,inf,1.0
7,vale5,ambv4,pearson,two-sided,1604,0.482919,"[0.44, 0.52]",1.738600e-94,5.688e+90,1.0
8,vale5,itub4,pearson,two-sided,1604,0.642838,"[0.61, 0.67]",1.064830e-187,5.347e+183,1.0
9,ambv4,itub4,pearson,two-sided,1604,0.488886,"[0.45, 0.53]",3.929767e-97,2.467e+93,1.0




*   Spearman



In [75]:
# correlacao
spearman_correlacao = dados_bolsa_filtrados.corr(method = 'spearman')
spearman_correlacao

<ipython-input-75-7ddd07dc76b4>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  spearman_correlacao = dados_bolsa_filtrados.corr(method = 'spearman')


,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.472143,0.644211,0.330002,0.531198
bbdc3,0.472143,1.000000,0.505002,0.419052,0.758946
vale5,0.644211,0.505002,1.000000,0.404373,0.574199
ambv4,0.330002,0.419052,0.404373,1.000000,0.449118
itub4,0.531198,0.758946,0.574199,0.449118,1.000000


In [76]:
# matriz ajustada
spearman_correlacao.style.background_gradient(cmap='coolwarm').set_precision(2)

<ipython-input-76-980577ec7d2f>:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  spearman_correlacao.style.background_gradient(cmap='coolwarm').set_precision(2)


,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.00,0.47,0.64,0.33,0.53
bbdc3,0.47,1.00,0.51,0.42,0.76
vale5,0.64,0.51,1.00,0.40,0.57
ambv4,0.33,0.42,0.40,1.00,0.45
itub4,0.53,0.76,0.57,0.45,1.00


In [77]:
# colunas para correlacionar
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']
# teste
pg.pairwise_corr(dados_bolsa_filtrados,
                columns = correl_columns,
                method = 'spearman')

,X,Y,method,alternative,n,r,CI95%,p-unc,power
0,petr4,bbdc3,spearman,two-sided,1604,0.472143,"[0.43, 0.51]",7.723804e-90,1.0
1,petr4,vale5,spearman,two-sided,1604,0.644211,"[0.61, 0.67]",9.494683e-189,1.0
2,petr4,ambv4,spearman,two-sided,1604,0.330002,"[0.29, 0.37]",4.669599e-42,1.0
3,petr4,itub4,spearman,two-sided,1604,0.531198,"[0.5, 0.57]",1.762927e-117,1.0
4,bbdc3,vale5,spearman,two-sided,1604,0.505002,"[0.47, 0.54]",1.513500e-104,1.0
5,bbdc3,ambv4,spearman,two-sided,1604,0.419052,"[0.38, 0.46]",3.166567e-69,1.0
6,bbdc3,itub4,spearman,two-sided,1604,0.758946,"[0.74, 0.78]",8.700692e-301,1.0
7,vale5,ambv4,spearman,two-sided,1604,0.404373,"[0.36, 0.44]",3.794800e-64,1.0
8,vale5,itub4,spearman,two-sided,1604,0.574199,"[0.54, 0.61]",2.395419e-141,1.0
9,ambv4,itub4,spearman,two-sided,1604,0.449118,"[0.41, 0.49]",1.896860e-80,1.0




*   Kendall



In [78]:
kendall_correlacao = dados_bolsa_filtrados.corr()
kendall_correlacao

<ipython-input-78-6ee957bafea4>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  kendall_correlacao = dados_bolsa_filtrados.corr()


,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.000000,0.539247,0.724023,0.392074,0.593834
bbdc3,0.539247,1.000000,0.592143,0.470529,0.778506
vale5,0.724023,0.592143,1.000000,0.482919,0.642838
ambv4,0.392074,0.470529,0.482919,1.000000,0.488886
itub4,0.593834,0.778506,0.642838,0.488886,1.000000


In [79]:
# matriz ajustada
kendall_correlacao.style.background_gradient(cmap='coolwarm').set_precision(2)

<ipython-input-79-2b9ff8799a26>:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  kendall_correlacao.style.background_gradient(cmap='coolwarm').set_precision(2)


,petr4,bbdc3,vale5,ambv4,itub4
petr4,1.00,0.54,0.72,0.39,0.59
bbdc3,0.54,1.00,0.59,0.47,0.78
vale5,0.72,0.59,1.00,0.48,0.64
ambv4,0.39,0.47,0.48,1.00,0.49
itub4,0.59,0.78,0.64,0.49,1.00


In [80]:
correl_columns = ['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']
correl_columns

['petr4', 'bbdc3', 'vale5', 'ambv4', 'itub4']

In [82]:
# teste
pg.pairwise_corr(dados_bolsa_filtrados,
                 columns = correl_columns,
                 method = 'kendall')

,X,Y,method,alternative,n,r,CI95%,p-unc,power
0,petr4,bbdc3,kendall,two-sided,1604,0.332691,"[0.29, 0.38]",1.358166e-88,1.0
1,petr4,vale5,kendall,two-sided,1604,0.471033,"[0.43, 0.51]",9.805061e-176,1.0
2,petr4,ambv4,kendall,two-sided,1604,0.231311,"[0.18, 0.28]",8.582460e-44,1.0
3,petr4,itub4,kendall,two-sided,1604,0.378755,"[0.34, 0.42]",2.531465e-114,1.0
4,bbdc3,vale5,kendall,two-sided,1604,0.358557,"[0.32, 0.4]",1.375615e-102,1.0
5,bbdc3,ambv4,kendall,two-sided,1604,0.294492,"[0.25, 0.34]",8.231269e-70,1.0
6,bbdc3,itub4,kendall,two-sided,1604,0.571953,"[0.54, 0.6]",7.030155e-258,1.0
7,vale5,ambv4,kendall,two-sided,1604,0.282942,"[0.24, 0.33]",1.246269e-64,1.0
8,vale5,itub4,kendall,two-sided,1604,0.411663,"[0.37, 0.45]",1.100820e-134,1.0
9,ambv4,itub4,kendall,two-sided,1604,0.318638,"[0.27, 0.36]",1.852468e-81,1.0
